## Prepare data reader and model execution context

In [2]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "account_abstraction"


# Select a model.
data_spec = ModelsDataSpec(root_path_prefix="blockbatch", models=[model_name])

# Select a block batch.
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="21770458:+1",
    root_paths_to_read=data_spec.input_root_paths,
)

# Construct readers
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)

# Show details for the batch we are processing.
pprint(readers[0])

# Ensure existence of data needed by the reader.
assert readers[0].inputs_ready

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-03-17 09:18:49 [info     ] querying markers at time range min: 2024-10-30 23:17:43, max: 2024-10-30 23:17:43 root_paths=['blockbatch/account_abstraction_prefilter/entrypoint_logs_v1', 'blockbatch/account_abstraction_prefilter/entrypoint_traces_v1'] filename=request.py lineno=146 process=31156
2025-03-17 09:18:49 [info     ] prepared 1 input batches.      filename=byblock.py lineno=92 process=31156
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain',
                                                      value='base'),
                                      PartitionColumn(name='dt',
                                                      value='2024-10-30')]),
           read_from=DataLocation.GCS,
           dataset_paths={'blockbatch/account_abstraction_prefilter/entrypoint_logs_v1': ['gs://oplabs-tools-data-sink/blockbatch/account_abstraction_prefilter/entrypoint_logs_v1/chain=base/dt=2024-10-30/000021770400.parquet'],
                          'blockbatch/account_ab

In [4]:
from op_analytics.datapipeline.models.code.account_abstraction.decoders import (
    register_4337_decoders,
)

from op_analytics.datapipeline.models.code.account_abstraction.abis import (
    HANDLE_OPS_FUNCTION_METHOD_ID_v0_6_0,
    HANDLE_OPS_FUNCTION_METHOD_ID_v0_7_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
)

register_4337_decoders(ctx)


# Decoded UserOperationEvent logs.
user_ops = auxiliary_templates["account_abstraction/useroperationevent_logs"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets[
            "blockbatch/account_abstraction_prefilter/entrypoint_logs_v1"
        ].as_subquery(),
    },
)

# Persist the prefiltered traces for performance gains.
prefiltered_traces = input_datasets[
    "blockbatch/account_abstraction_prefilter/entrypoint_traces_v1"
].create_table()

# Traces initiated on behalf of the UserOperationEvent sender
entrypoint_traces = auxiliary_templates[
    "account_abstraction/enriched_entrypoint_traces"
].create_view(
    duckdb_context=ctx,
    template_parameters={
        "prefiltered_traces": prefiltered_traces,
        "uops": user_ops,
        "method_id_v6": INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
        "method_id_v7": INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
        "handle_ops_v6": HANDLE_OPS_FUNCTION_METHOD_ID_v0_6_0,
        "handle_ops_v7": HANDLE_OPS_FUNCTION_METHOD_ID_v0_7_0,
    },
)

# Data Quality Checks
errors = []
for name, val in auxiliary_templates.items():
    if "data_quality_check" in name:
        errors.extend(val.run_as_data_quality_check(duckdb_context=ctx))
# if errors:
#     raise Exception("\n\n".join([name] + [str(_) for _ in errors]))
# else:
#     print("Data Quality OK")



2025-03-17 09:20:21 [warning  ] duckdb function is already registered: decode_useroperationevent filename=register.py lineno=22 process=31156
2025-03-17 09:20:21 [warning  ] duckdb function is already registered: decode_innerhandleop filename=register.py lineno=22 process=31156
2025-03-17 09:20:21 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=290 process=31156
2025-03-17 09:20:21 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=31156 template=account_abstraction/useroperationevent_logs
2025-03-17 09:20:23 [info     ] duck db size: 42.2MB           filename=client.py lineno=40 process=31156
2025-03-17 09:20:23 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=290 process=31156
2025-03-17 09:20:24 [info     ] created table/view blockbatch_account_abstraction_prefilter_entrypoint_traces_v1_tbl filename=client.py lineno=229 process=31156
2025-03-17 09:20:24 [info     ] duck db siz

In [6]:
errors

[{'error': 'Unsatisfied JOIN for UserOperationEvent <> EntryPoint traces',
  'block_number': 21770787,
  'transaction_hash': '0xf2502957b2099bee9864eaaaafd5386c9d6de46607ebf64fdbff25df92101a9e',
  'log_index': 107,
  'sender_logs': '0xa2085974c51a2611f416aa939f6bc633a8b2b2b4',
  'sender_innerhandleop': None},
 {'error': 'Unsatisfied JOIN for UserOperationEvent <> EntryPoint traces',
  'block_number': 21770616,
  'transaction_hash': '0xc9df13a805a4db376da7518daaf16103c53a3521d155089b51f485f6b56c1302',
  'log_index': 163,
  'sender_logs': '0x7db44ee78be8fd5606e1db72970df149463bf0ca',
  'sender_innerhandleop': None},
 {'error': 'Unsatisfied JOIN for UserOperationEvent <> EntryPoint traces',
  'block_number': 21770460,
  'transaction_hash': '0xf18ff9174ae567b21e7da4ffa9f051061b9c9af8bf857408a2076d1876fadf51',
  'log_index': 313,
  'sender_logs': '0xc5f18182ad6acf1718f0bf88a5449802e0a813bc',
  'sender_innerhandleop': None},
 {'error': 'Unsatisfied JOIN for UserOperationEvent <> EntryPoint t

In [ ]:
ctx.client.sql("SHOW TABLES")

┌───────────────────────────────────────────────────────────────────┐
│                               name                                │
│                              varchar                              │
├───────────────────────────────────────────────────────────────────┤
│ account_abstraction__enriched_entrypoint_traces                   │
│ account_abstraction__useroperationevent_logs                      │
│ blockbatch_account_abstraction_prefilter_entrypoint_traces_v1_tbl │
└───────────────────────────────────────────────────────────────────┘

In [8]:
# The logs show that there were 2 user ops

ctx.client.sql(f"""
    SELECT *
    FROM {user_ops}
    WHERE transaction_hash =  '0x90d69e803dcef7e4bcdde5b19fe4f22cd10c097ec30208c1b01d0a7b43818388'
""")

┌────────────┬─────────┬──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬───────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────┬─────────┬───────────────┬───────────────┐
│     dt     │  chain  │ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │ log_index │              contract_address              │                             userophash                             │                   sender                   │                 paymaster                  │  nonce  │ success │ actualGasCost │ actual

In [ ]:
# The traces only have 1 huser op

ctx.client.sql(f"""
    SELECT 
        transaction_hash, 
        trace_root, 
        trace_address, 
        innerhandleop_trace_address, 
        is_from_sender, 
        is_innerhandleop,
        userop_sender,
        useropevent_success,
        status,
        error,
        from_address, 
        to_address
    FROM {entrypoint_traces} 
    WHERE
        transaction_hash = '0x90d69e803dcef7e4bcdde5b19fe4f22cd10c097ec30208c1b01d0a7b43818388'
""").show(max_rows=100)

┌────────────────────────────────────────────────────────────────────┬────────────┬───────────────┬─────────────────────────────┬────────────────┬──────────────────┬────────────────────────────────────────────┬─────────────────────┬────────┬─────────┬────────────────────────────────────────────┬────────────────────────────────────────────┐
│                          transaction_hash                          │ trace_root │ trace_address │ innerhandleop_trace_address │ is_from_sender │ is_innerhandleop │               userop_sender                │ useropevent_success │ status │  error  │                from_address                │                 to_address                 │
│                              varchar                               │   int32    │    varchar    │           varchar           │    boolean     │     boolean      │                  varchar                   │       boolean       │ int64  │ varchar │                  varchar                   │                  va

In [11]:
# Pre-filtered traces

ctx.client.sql(f"""
    SELECT 
        * EXCLUDE(input, output)
    FROM {prefiltered_traces} 
    WHERE transaction_hash = '0x90d69e803dcef7e4bcdde5b19fe4f22cd10c097ec30208c1b01d0a7b43818388'
    ORDER BY TRY_CAST(trace_root AS INT), trace_address
""").show(max_rows=100)

HANDLE_OPS_FUNCTION_METHOD_ID_v0_6_0 = "0x1fad948c"
INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0 = "0x1d732756"

HANDLE_OPS_FUNCTION_METHOD_ID_v0_7_0 = "0x765e827f"
INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0 = "0x0042dc53"


┌──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────┬────────────┬────────────┬─────────────┬─────────┬──────────┬───────────┬───────────────┬─────────┬────────┬────────────┬────────────┬─────────┬────────────┬────────────┐
│ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │                from_address                │                 to_address                 │ value_lossless │ trace_type │ call_type  │ reward_type │   gas   │ gas_used │ subtraces │ trace_address │  error  │ status │ trace_root │ method_id  │  chain  │     dt     │ method_id  │
│  int32   │ varch

In [39]:
import requests

response = requests.post(
    url="https://docs-demo.base-mainnet.quiknode.pro/",
    headers={"Content-Type": "application/json"},
    json={
        "method": "debug_traceTransaction",
        "params": [
            "0x90d69e803dcef7e4bcdde5b19fe4f22cd10c097ec30208c1b01d0a7b43818388",
            {"tracer": "callTracer"},
        ],
        "id": 1,
        "jsonrpc": "2.0",
    },
)

response_data = response.json()["result"]

# curl https://docs-demo.base-mainnet.quiknode.pro/ \
# -X POST \
# -H "Content-Type: application/json" \
# --data '{"method":"debug_traceTransaction","params":["0x2d3750f7944818e1a4e6b897b6f6ea4d975fcf46720990eb4eda0c00d926a8fd", {"tracer": "callTracer"}], "id":1,"jsonrpc":"2.0"}'


200

In [14]:
response

<Response [200]>

In [40]:
import json
from typing import Dict, List, Any

import polars as pl

def flatten_traces(calls: List[Dict[str, Any]], parent_trace_address: List[int]) -> List[Dict[str, Any]]:
    """
    Recursively flatten the nested call traces structure while preserving the trace address.
    
    Args:
        calls: List of call traces
        parent_trace_address: The trace address of the parent call
        
    Returns:
        List of flattened call traces with trace_address field
    """
    
    flattened_traces = []
    
    for i, call in enumerate(calls):
        

        
        # Calculate the current trace address
        current_trace_address = parent_trace_address + [i]
        
        # Add trace_address to the call        
        row = {
            "trace_address": current_trace_address,
            "from": call["from"],
            "method_id": call["input"][:10],
        }
        
        # Extract nested calls before removing them from the copy
        nested_calls = call.get('calls', [])        
        # Add the current call to the flattened list
        flattened_traces.append(row)
        
        # Recursively process nested calls
        if nested_calls:
            flattened_traces.extend(flatten_traces(nested_calls, current_trace_address))
    
    return flattened_traces

# Load the base_missing_traces.json file
with open('base_missing_traces.json', 'r') as f:
    trace_data = json.load(f)

# Flatten the traces
rows = flatten_traces(response_data.get("calls", []), parent_trace_address=[])
for row in rows:
    row["trace_address"] = ",".join([str(_) for _ in row["trace_address"]])

pldf = pl.DataFrame(rows)
ctx.client.sql("SELECT * FROm pldf").show(max_rows=100)



┌───────────────┬────────────────────────────────────────────┬────────────┐
│ trace_address │                    from                    │ method_id  │
│    varchar    │                  varchar                   │  varchar   │
├───────────────┼────────────────────────────────────────────┼────────────┤
│ 0             │ 0x5ff137d4b0fdcd49dca30c7cf57e578a026d2789 │ 0x570e1a36 │
│ 0,0           │ 0x7fc98430eaedbb6070b35b39d798725049088348 │ 0x5fbfb9cf │
│ 0,0,0         │ 0x15ba39375ee2ab563e8873c8390be6f2e2f50232 │ 0x60806040 │
│ 0,0,0,0       │ 0xcf2ed5279d4f68d18ddd561772e67b700e10b063 │ 0xc4d66de8 │
│ 1             │ 0x5ff137d4b0fdcd49dca30c7cf57e578a026d2789 │ 0x3a871cdd │
│ 1,0           │ 0xcf2ed5279d4f68d18ddd561772e67b700e10b063 │ 0x3a871cdd │
│ 1,0,0         │ 0xcf2ed5279d4f68d18ddd561772e67b700e10b063 │ 0xe3180ea3 │
│ 2             │ 0x5ff137d4b0fdcd49dca30c7cf57e578a026d2789 │ 0xf465c77e │
│ 2,0           │ 0xa270ef92c1e11f1c1f95753c2e56801e8125fa83 │ 0x439a4780 │
│ 3         